In [33]:
import pandas as pd
import numpy as np
from nltk.tokenize import (word_tokenize)
from nltk import WordNetLemmatizer
from gensim.models import Word2Vec
import gensim.downloader
import spacy
from sklearn.mixture import GaussianMixture

In [34]:
glove_vectors = gensim.downloader.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [4]:
spacy_object = spacy.load("en_core_web_sm")

In [2]:
df = pd.read_csv("./cnn_dailymail/train.csv")
df.drop(["id"], axis=1, inplace=True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143557 entries, 0 to 143556
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   article     143557 non-null  object
 1   highlights  143557 non-null  object
 2   cleaned     143557 non-null  object
dtypes: object(3)
memory usage: 3.3+ MB


In [5]:
df = df[:len(df)//2+1]

In [7]:
def remove_splchars(text):
    spl_chars = ["/", ":", "@", "*", "[", "]", "(", ")", "+", "-", "=", "|"]
    res = ""
    for i in text:
        if i in spl_chars:
            continue
        else:
            res += i
    return res

In [10]:
df.loc[:,"cleaned"]= ""
df.head()

/var/folders/_p/8jqb34k56f1b983tk9xx_lzr0000gn/T/ipykernel_88786/3243560449.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"cleaned"]= ""


,article,highlights,cleaned
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ...",
1,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...,
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t...",
3,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...,
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...,


In [11]:
df_size = df["article"].size

for i in range (0, df_size):
    temp = df["article"][i]
    cleaned = remove_splchars(temp)
    df["cleaned"][i] = cleaned

/var/folders/_p/8jqb34k56f1b983tk9xx_lzr0000gn/T/ipykernel_88786/3260971211.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cleaned"][i] = cleaned


In [13]:
df.head()

,article,highlights,cleaned
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ...","By . Associated Press . PUBLISHED . 1411 EST, ..."
1,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...,CNN Ralph Mata was an internal affairs lieute...
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t...",A drunk driver who killed a young woman in a h...
3,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...,CNN With a breezy sweep of his pen President ...
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...,Fleetwood are the only team still to have a 10...


In [14]:
df.to_pickle("cleaned.pkl")

In [2]:
df = pd.read_pickle("./cleaned.pkl")

In [15]:
df = df.drop("article", axis=1)

/var/folders/_p/8jqb34k56f1b983tk9xx_lzr0000gn/T/ipykernel_88786/336636277.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop("article", axis=1, inplace=True)


In [16]:
df.head()

,highlights,cleaned
0,"Bishop John Folda, of North Dakota, is taking ...","By . Associated Press . PUBLISHED . 1411 EST, ..."
1,Criminal complaint: Cop used his role to help ...,CNN Ralph Mata was an internal affairs lieute...
2,"Craig Eccleston-Todd, 27, had drunk at least t...",A drunk driver who killed a young woman in a h...
3,Nina dos Santos says Europe must be ready to a...,CNN With a breezy sweep of his pen President ...
4,Fleetwood top of League One after 2-0 win at S...,Fleetwood are the only team still to have a 10...


In [19]:
df["tokens"] = ""

/var/folders/_p/8jqb34k56f1b983tk9xx_lzr0000gn/T/ipykernel_88786/1597212705.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["tokens"] = ""


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143557 entries, 0 to 143556
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   highlights  143557 non-null  object
 1   cleaned     143557 non-null  object
 2   tokens      143557 non-null  object
dtypes: object(3)
memory usage: 3.3+ MB


In [21]:
df_size = df["cleaned"].size

for i in range (0, df_size):
    temp = df.loc[i,"cleaned"]
    tokenised = word_tokenize(temp, 'english', preserve_line=True)
    df["tokens"][i] = tokenised


/var/folders/_p/8jqb34k56f1b983tk9xx_lzr0000gn/T/ipykernel_88786/3644086015.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["tokens"][i] = tokenised


In [5]:
df.to_pickle("tokened.pkl")

NameError: name 'df' is not defined


def batching(df):
    batch_size = 1000
    temp = 0
    steps = np.ceil((df["cleaned"].size)/batch_size)
    while(steps>=0):
        df_temp = df.iloc[temp*batch_size:(temp+1)*batch_size, 1]
        yield df_temp
        temp+=1
        steps-=1 

count = 0

for i in batching(df):
    temp = i.transform(lambda x: word_tokenize(x, "english", preserve_line=True))
    # temp = word_tokenize(i, "english", preserve_line=True)
    df.iloc[count*1000:(count+1)*1000, 2] = temp 
    count += 1

In [6]:
df = pd.read_pickle("tokened.pkl")

In [7]:
df["lemmas"] = ""
df_size = df["cleaned"].size

new_df = df.loc[0:10,"tokens"].transform(lambda x: [spacy_object(token)[0].lemma_ for token in x ])

# for i in range (0, df_size):
#     temp = df.loc[i,"cleaned"]
#     l = spacy_object("canceled")
#     print(l[0].lemma_)
#     for i in l:
#         print(i,i.lemma_)
#     break
#     df["lemmas"][i] = tokenised

In [8]:
df.loc[0,"cleaned"]

"By . Associated Press . PUBLISHED . 1411 EST, 25 October 2013 .  . UPDATED . 1536 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda pictured of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through contaminated food while attending a conference for newly ordained bishops

In [12]:
article = " ".join(new_df[0])

In [49]:
tokenised = word_tokenize(article, 'english', preserve_line=True)

In [52]:
model = Word2Vec(tokenised, vector_size=100, window=4)
model.train(tokenised,total_examples=1,epochs=1)

(147, 965)

In [44]:
model.train(df.loc[0,"tokens"],total_examples = 1, epochs=1)

(181, 1005)

In [48]:
glove_vectors.most_similar('health')

[('education', 0.9281142354011536),
 ('financial', 0.9185183048248291),
 ('benefits', 0.9174034595489502),
 ('medical', 0.9139702916145325),
 ('healthcare', 0.9132720828056335),
 ('research', 0.9052173495292664),
 ('investment', 0.9006076455116272),
 ('daily', 0.8977178335189819),
 ('aid', 0.8971127271652222),
 ('industry', 0.896756112575531)]

In [53]:
model.wv["bishop"]

KeyError: "Key 'bishop' not present"

In [54]:
from sklearn.feature_extraction.text import CountVectorizer

In [59]:
vectorizer = CountVectorizer()

vectorizer.fit(df.loc[0, 'cleaned'][0])

bow_model = vectorizer.transform(df.loc[0:10, 'tokens'])

ValueError: Iterable over raw text documents expected, string object received.

In [ ]:
print(bow_model)